In [1]:
import time, random
from multiprocessing import Pool   

In [4]:
# 这个类就是个筐，各种参数都可以往里装
class TaskConfig:
    def __init__(self, id, seconds):
        self.id = id
        # 运行参数
        self.seconds = seconds
# 运行结果（这里只考虑正常结果，也可以加上异常结果，用一个bool变量来区分成功和失败）
class TaskResult:
    def __init__(self, id, result):
        self.id = id
        self.result = result    
# 模拟耗时很长的任务
def run_sim(config):
    time.sleep(config.seconds)
    result = 'Task %d finished after %d seconds' % (config.id, config.seconds) 
    return TaskResult(config.id, result)
# 生成配置的函数，每个任务随机取参数1~10   
def gen_config(id):
    time_length = random.randint(1,7)
    print('Task %d need %d seconds to finish' % (id, time_length) )
    return TaskConfig(id, time_length)
def run_a_round(configs):
    worker_pool = Pool(processes=len(configs))  
    async_results = []
    for config in configs:
        async_result = worker_pool.apply_async(run_sim, (config,))
        async_results.append(async_result)
    # 不再允许提交任务
    worker_pool.close()
    # 设置5秒超时，主进程负责每1秒检查一次任务是否完成
    TIMEOUT = 10
    for i in range(TIMEOUT):
        time.sleep(1)
        all_ready = all([async_result.ready() for async_result in async_results])
        if all_ready:
            break          
    # 要么全部完成，要么超时
    task_results = []
    for i, async_result in enumerate(async_results):
        if async_result.ready():
            task_result = async_result.get()
            print(task_result.result)
            task_results.append(task_result)
        else:
            print('Task %d timeout' % i)    
    # 关闭进程池释放资源
    worker_pool.terminate()  
    return task_results

In [5]:
    
if __name__ == '__main__': 
    # 初始化10个任务
    task_configs = [gen_config(i) for i in range(3)]
    
    # 跑到没有任务可跑为止，最多跑三轮
    round_cnt = 1
    while len(task_configs) > 0 and round_cnt <= 3:
        print('--- round %d ---' % round_cnt)
        task_results = run_a_round(task_configs)
        # 根据结果更新下一轮参数
        task_configs = [gen_config(task_result.id) for task_result in task_results]
        
        round_cnt += 1
    
    

Task 0 need 6 seconds to finish
Task 1 need 4 seconds to finish
Task 2 need 3 seconds to finish
--- round 1 ---
Task 0 timeout
Task 1 timeout
Task 2 timeout
